In [53]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import nltk 
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import requests

#Loading Data

In [21]:
anime_df = pd.read_csv('/content/drive/MyDrive/anime rec/animes.csv')

In [22]:
anime_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...


In [23]:
anime_df.shape

(19311, 12)

In [24]:
#removing duplicates
anime_df.drop_duplicates(inplace=True)
anime_df.shape

(16368, 12)

#Finding 5 most popular anime data

In [25]:
popularanimes = anime_df[anime_df['popularity'] <= 6][['title','score','img_url']] 
pickle.dump(popularanimes,open('popularanimes.pkl','wb'))

In [26]:
popularanimes.head()

,title,score,img_url
3,Fullmetal Alchemist: Brotherhood,9.23,https://cdn.myanimelist.net/images/anime/1223/...
16,One Punch Man,8.68,https://cdn.myanimelist.net/images/anime/12/76...
140,Tokyo Ghoul,7.93,https://cdn.myanimelist.net/images/anime/5/644...
681,Shingeki no Kyojin,8.47,https://cdn.myanimelist.net/images/anime/10/47...
740,Death Note,8.65,https://cdn.myanimelist.net/images/anime/9/945...


#Here, we filter animes which have ratings higher than 7 to reduce data complexity. 

In [27]:
anime_df = anime_df[anime_df['score']>7]

Scores are classified to categories which are later used as tags. 

In [28]:
def getScore(x):
  if x>9: 
    return "veryhighscore"
  elif x>8:
    return "highscore"
  elif x>7:
    return "goodscore"
  else:
    return "okscore"
anime_df['rating'] = anime_df['score'].apply(getScore)
anime_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,rating
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,"['Comedy', 'Sports', 'Drama', 'School', 'Shoun...","Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,highscore
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,"['Drama', 'Music', 'Romance', 'School', 'Shoun...","Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,highscore
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,"['Sci-Fi', 'Adventure', 'Mystery', 'Drama', 'F...","Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,highscore
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...","['Action', 'Military', 'Adventure', 'Comedy', ...","Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,veryhighscore
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,"['Action', 'Mystery', 'Supernatural', 'Vampire']","Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,highscore


The genres are in string format and needs to be converted to list. The list of genres are then joined to space separated strings

In [29]:
anime_df['genre'].iloc[0]

"['Comedy', 'Sports', 'Drama', 'School', 'Shounen']"

In [30]:
import ast
def convert2list(x):
  return ast.literal_eval(x)
anime_df['genre'] = anime_df['genre'].apply(convert2list)
anime_df['genre']

0                 [Comedy, Sports, Drama, School, Shounen]
1                 [Drama, Music, Romance, School, Shounen]
2             [Sci-Fi, Adventure, Mystery, Drama, Fantasy]
3        [Action, Military, Adventure, Comedy, Drama, M...
4                 [Action, Mystery, Supernatural, Vampire]
                               ...                        
19002    [Action, Comedy, Super Power, Martial Arts, Sh...
19003                [Slice of Life, Comedy, Supernatural]
19004                     [Slice of Life, Comedy, Shounen]
19005                                             [Action]
19006      [Comedy, Drama, Romance, School, Slice of Life]
Name: genre, Length: 4224, dtype: object

In [31]:
anime_df['genre'] = anime_df['genre'].apply(lambda x: ' '.join(x))
anime_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,rating
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,Comedy Sports Drama School Shounen,"Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,highscore
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,Drama Music Romance School Shounen,"Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,highscore
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,Sci-Fi Adventure Mystery Drama Fantasy,"Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,highscore
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",Action Military Adventure Comedy Drama Magic F...,"Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,veryhighscore
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,Action Mystery Supernatural Vampire,"Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,highscore


Tags column is created by combining the necessary features

In [32]:
anime_df['tags'] = anime_df['rating'] + ' ' + anime_df['synopsis'] + ' ' + anime_df['genre'] 
anime_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,rating,tags
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,Comedy Sports Drama School Shounen,"Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,highscore,highscore Following their participation at the...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,Drama Music Romance School Shounen,"Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,highscore,highscore Music accompanies the path of the hu...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,Sci-Fi Adventure Mystery Drama Fantasy,"Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,highscore,highscore The Abyss—a gaping chasm stretching ...
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",Action Military Adventure Comedy Drama Magic F...,"Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,veryhighscore,"veryhighscore ""In order for something to be ob..."
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,Action Mystery Supernatural Vampire,"Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,highscore,highscore After helping revive the legendary v...


In [33]:
anime_df['tags'].iloc[0]

"highscore Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead.  \r\n \r\nWhen they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but also come up with new attacks that would strengthen them. Moreover, Hinata and Kageyama attempt to devise a more powerful weapon, one that could possibly break the sturdiest of blocks.  \r\n \r\nFacing what may be their last chance at victory before the senior players graduate, the members of Karasuno's volleyball team must learn to settle their differences and train harder than ever if they hope to overcome formidable opponents old and new—including their archrival Aoba Jousai and its wo

Tags are converted to lower case to avoid duplicacy. 

In [34]:
anime_df['tags'] = anime_df['tags'].apply(lambda x: str(x).lower()) 
anime_df.head()

,uid,title,synopsis,genre,aired,episodes,members,popularity,ranked,score,img_url,link,rating,tags
0,28891,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,Comedy Sports Drama School Shounen,"Oct 4, 2015 to Mar 27, 2016",25.0,489888,141,25.0,8.82,https://cdn.myanimelist.net/images/anime/9/766...,https://myanimelist.net/anime/28891/Haikyuu_Se...,highscore,highscore following their participation at the...
1,23273,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,Drama Music Romance School Shounen,"Oct 10, 2014 to Mar 20, 2015",22.0,995473,28,24.0,8.83,https://cdn.myanimelist.net/images/anime/3/671...,https://myanimelist.net/anime/23273/Shigatsu_w...,highscore,highscore music accompanies the path of the hu...
2,34599,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,Sci-Fi Adventure Mystery Drama Fantasy,"Jul 7, 2017 to Sep 29, 2017",13.0,581663,98,23.0,8.83,https://cdn.myanimelist.net/images/anime/6/867...,https://myanimelist.net/anime/34599/Made_in_Abyss,highscore,highscore the abyss—a gaping chasm stretching ...
3,5114,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",Action Military Adventure Comedy Drama Magic F...,"Apr 5, 2009 to Jul 4, 2010",64.0,1615084,4,1.0,9.23,https://cdn.myanimelist.net/images/anime/1223/...,https://myanimelist.net/anime/5114/Fullmetal_A...,veryhighscore,"veryhighscore ""in order for something to be ob..."
4,31758,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,Action Mystery Supernatural Vampire,"Jan 6, 2017",1.0,214621,502,22.0,8.83,https://cdn.myanimelist.net/images/anime/3/815...,https://myanimelist.net/anime/31758/Kizumonoga...,highscore,highscore after helping revive the legendary v...


In [35]:
anime_df.shape

(4224, 14)

The words in tags are stemmed to handle similar words. 

In [36]:
ps = PorterStemmer()

In [37]:
def stemming(sentence):
  x = []
  for word in sentence.split():
    x.append(ps.stem(word))
  return " ".join(x)

In [38]:
anime_df['tags'] = anime_df['tags'].apply(stemming)

The tags are vectorized.

In [39]:
vectorizer = CountVectorizer(max_features=6000,stop_words='english')
vectortags = vectorizer.fit_transform(anime_df['tags']).toarray()

In [40]:
vectortags

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [41]:
vectorizer.get_feature_names()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['000',
 '01',
 '09',
 '10',
 '100',
 '10th',
 '11',
 '11th',
 '12',
 '12th',
 '13',
 '13th',
 '14',
 '14th',
 '15',
 '15th',
 '16',
 '16th',
 '17',
 '17th',
 '18',
 '18th',
 '19',
 '1945',
 '1995',
 '1998',
 '1999',
 '19th',
 '1st',
 '20',
 '2005',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2039',
 '20th',
 '21',
 '21st',
 '22',
 '22nd',
 '23',
 '24',
 '24th',
 '25',
 '25th',
 '26',
 '27',
 '29',
 '2nd',
 '30',
 '300',
 '31',
 '38',
 '3d',
 '3rd',
 '40',
 '48',
 '4th',
 '50',
 '500',
 '5th',
 '6th',
 '765',
 '7th',
 '8th',
 '999',
 '9th',
 'abandon',
 'abduct',
 'abe',
 'abel',
 'abeno',
 'abh',
 'abil',
 'abilities',
 'ability',
 'abl',
 'abnorm',
 'aboard',
 'abov',
 'abras',
 'abroad',
 'abrupt',
 'abruptli',
 'absence',
 'absent',
 'absolut',
 'absorb',
 'abund',
 'abus',
 'abyss',
 'academ',
 'academi',
 'academia',
 'academy',
 'acceler',
 'accept',
 'accepts',
 'access',
 'accid',
 'accident',
 'acclaim',
 'acco

Similarity Matrix is obtained by finding the cosinesimilarity between the vectortags of each animes. 

In [42]:
similarityMatrix = cosine_similarity(vectortags)

In [43]:
similarityMatrix[0]

array([1.        , 0.10196784, 0.080708  , ..., 0.08479507, 0.0608475 ,
       0.07389222])

The function finds the similar animes to the given anime by finding the animes with maximum cosinesimilarity. 

In [44]:
def recommendAnime(animeName):
  givenAnimeIndex = anime_df[anime_df['title']==animeName].index[0]
  similarity = similarityMatrix[givenAnimeIndex]
  animeList = sorted(list(enumerate(similarity)),reverse=True, key = lambda x:x[1])[1:6]
  data = []
  for anime in animeList:
    item = []
    tempdf = anime_df.iloc[anime[0]]
    item.append(tempdf['title'])
    item.append(tempdf['score'])
    item.append(tempdf['img_url'])
    data.append(item)
  return data

In [45]:
recommendAnime("Naruto")

[['Boruto: Naruto the Movie',
  7.71,
  'https://cdn.myanimelist.net/images/anime/4/78280.jpg'],
 ['Naruto: Shippuuden',
  8.2,
  'https://cdn.myanimelist.net/images/anime/5/17407.jpg'],
 ['Naruto: Shippuuden Movie 5 - Blood Prison',
  7.56,
  'https://cdn.myanimelist.net/images/anime/13/41403.jpg'],
 ['Naruto SD: Rock Lee no Seishun Full-Power Ninden',
  7.22,
  'https://cdn.myanimelist.net/images/anime/13/36475.jpg'],
 ['Naruto: Shippuuden Movie 6 - Road to Ninja',
  7.77,
  'https://cdn.myanimelist.net/images/anime/6/51863.jpg']]

In [46]:
pickle.dump(similarityMatrix,open('similarityMatrix.pkl','wb'))
pickle.dump(anime_df,open('anime_df.pkl','wb'))

The review of animes are obtained using the anime uid through jikan API. Sentiment analysis is runned on the reviews using pretrained hugging face model.

In [47]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.1 MB/s 
     |████████████████████████████████| 7.6 MB 40.3 MB/s 
     |████████████████████████████████| 182 kB 45.2 MB/s 


In [49]:
from transformers import pipeline
import torch 
import torch.nn.functional as F 

In [51]:
sentiment_analysis = pipeline("sentiment-analysis",model="siebert/sentiment-roberta-large-english")

In [52]:
def sentimentAnalysis(uid):
  response = requests.get(f'https://api.jikan.moe/v4/anime/{uid}/reviews')
  data =response.json()
  r = data.get('data')[0].get('review')
  print(r)
  print(sentiment_analysis(r))

In [54]:
sentimentAnalysis(223)

Ah, Dragonball. The beginning of one of the longest and arguably most popular anime franchise on both sides of the pacific. Unfortunately, this anime seems to be shunned by a few based on its affiliation with its sequels. That's a shame, really. One huge reason why this anime holds its own is that the entire franchise is in its infancy. There are no Super Saiya-jin stages to worry about, and the fights aren&rsquo;t stretched out to their barely tolerable limits. The various quests that the young Z-Senshi embark on are just that: quests. There are no interstellar enemies to fight (early on), and the majority ofthe time spent is on the team simply trying to find the dragonballs. The comedic element in Dragonball shines just as brightly and flows smoothly with the story. The characters have such a genuinely portrayed chemisty between each other. Dragonball never seems to take itself too seriously, allowing the viewer to sit back and enjoy the ride.

For a twenty-year-old series, Dragonbal